<a href="https://colab.research.google.com/github/Foluwa/ml-zoomcamp-2024/blob/main/02-regression/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/laptops.csv

--2024-10-07 00:39:37--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/laptops.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 298573 (292K) [text/plain]
Saving to: ‘laptops.csv’

laptops.csv         100%[===================>] 291.58K  --.-KB/s    in 0.04s   

2024-10-07 00:39:37 (6.63 MB/s) - ‘laptops.csv’ saved [298573/298573]



In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('laptops.csv')

In [4]:
df.head()

,Laptop,Status,Brand,Model,CPU,RAM,Storage,Storage type,GPU,Screen,Touch,Final Price
0,ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core...,New,Asus,ExpertBook,Intel Core i5,8,512,SSD,NaN,15.6,No,1009.00
1,Alurin Go Start Intel Celeron N4020/8GB/256GB ...,New,Alurin,Go,Intel Celeron,8,256,SSD,NaN,15.6,No,299.00
2,ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core...,New,Asus,ExpertBook,Intel Core i3,8,256,SSD,NaN,15.6,No,789.00
3,MSI Katana GF66 12UC-082XES Intel Core i7-1270...,New,MSI,Katana,Intel Core i7,16,1000,SSD,RTX 3050,15.6,No,1199.00
4,HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB...,New,HP,15S,Intel Core i5,16,512,SSD,NaN,15.6,No,669.01


In [5]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [6]:
df.head()

,laptop,status,brand,model,cpu,ram,storage,storage_type,gpu,screen,touch,final_price
0,ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core...,New,Asus,ExpertBook,Intel Core i5,8,512,SSD,NaN,15.6,No,1009.00
1,Alurin Go Start Intel Celeron N4020/8GB/256GB ...,New,Alurin,Go,Intel Celeron,8,256,SSD,NaN,15.6,No,299.00
2,ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core...,New,Asus,ExpertBook,Intel Core i3,8,256,SSD,NaN,15.6,No,789.00
3,MSI Katana GF66 12UC-082XES Intel Core i7-1270...,New,MSI,Katana,Intel Core i7,16,1000,SSD,RTX 3050,15.6,No,1199.00
4,HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB...,New,HP,15S,Intel Core i5,16,512,SSD,NaN,15.6,No,669.01


In [10]:
df = df[['ram','storage', 'screen', 'final_price']]

In [8]:
df

,ram,storage,screen,final_price
0,8,512,15.6,1009.00
1,8,256,15.6,299.00
2,8,256,15.6,789.00
3,16,1000,15.6,1199.00
4,16,512,15.6,669.01
...,...,...,...,...
2155,16,1000,17.3,2699.99
2156,16,1000,17.3,2899.99
2157,32,1000,17.3,3399.99
2158,16,1000,13.4,1899.99


# Question 1. Missing values

In [11]:
# Find columns with missing values
missing_values = df.isnull().sum()

# Filter to display only columns with missing values
missing_columns = missing_values[missing_values > 0]
missing_columns


,0
screen,4


# Question 2. Median for RAM

In [14]:
# Calculate the median of the 'RAM' column
median_ram = df['ram'].median()
print("Median RAM:", median_ram)


Median RAM: 16.0


# Question 3. Filling NAs

- We need to deal with missing values for the column from Q1.
- We have two options: fill it with 0 or with the mean of this variable.
- Try both options. For each, train a linear regression model without regularization using the code from the lessons.
- For computing the mean, use the training only!
- Use the validation dataset to evaluate the models and compare the RMSE of each option.
- Round the RMSE scores to 2 decimal digits using round(score, 2)
- Which option gives better RMSE?

In [16]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Assuming 'Final Price' is the target variable
target_variable = 'final_price'
missing_column = 'ram'

# Split the dataset into features and target
X = df.drop(columns=[target_variable])
y = df[target_variable]

# Convert categorical variables into dummy variables
X = pd.get_dummies(X, drop_first=True)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Option 1: Fill all missing values with 0
X_train_zero = X_train.fillna(0)
X_val_zero = X_val.fillna(0)

# Option 2: Fill all missing values with the mean of the training data
X_train_mean = X_train.fillna(X_train.mean())
X_val_mean = X_val.fillna(X_train.mean())

# Train the model for Option 1 (filled with 0)
model_zero = LinearRegression()
model_zero.fit(X_train_zero, y_train)
y_pred_zero = model_zero.predict(X_val_zero)
rmse_zero = round(np.sqrt(mean_squared_error(y_val, y_pred_zero)), 2)

# Train the model for Option 2 (filled with mean)
model_mean = LinearRegression()
model_mean.fit(X_train_mean, y_train)
y_pred_mean = model_mean.predict(X_val_mean)
rmse_mean = round(np.sqrt(mean_squared_error(y_val, y_pred_mean)), 2)

# Output RMSE scores for comparison
print("RMSE (fill with 0):", rmse_zero)
print("RMSE (fill with mean):", rmse_mean)

# Compare and determine which option has the lower RMSE
if rmse_zero < rmse_mean:
    print("Filling with 0 gives a better RMSE.")
else:
    print("Filling with mean gives a better RMSE.")


RMSE (fill with 0): 624.88
RMSE (fill with mean): 625.51
Filling with 0 gives a better RMSE.


# Question 4. Regularization

In [17]:
from sklearn.linear_model import Ridge

# Define the list of regularization strengths to try
regularization_values = [0, 0.01, 0.1, 1, 5, 10, 100]

# Prepare the training and validation sets, filling missing values with 0
X_train_zero = X_train.fillna(0)
X_val_zero = X_val.fillna(0)

# Dictionary to store RMSE scores for each regularization value
rmse_scores = {}

# Loop over each regularization value, train the model, and evaluate RMSE
for r in regularization_values:
    model = Ridge(alpha=r)
    model.fit(X_train_zero, y_train)
    y_pred = model.predict(X_val_zero)
    rmse = round(np.sqrt(mean_squared_error(y_val, y_pred)), 2)
    rmse_scores[r] = rmse
    print(f"RMSE for r={r}: {rmse}")

# Find the regularization value with the best (lowest) RMSE
best_r = min(rmse_scores, key=rmse_scores.get)
best_rmse = rmse_scores[best_r]
print(f"\nBest r: {best_r} with RMSE: {best_rmse}")


RMSE for r=0: 624.88
RMSE for r=0.01: 624.88
RMSE for r=0.1: 624.88
RMSE for r=1: 624.88
RMSE for r=5: 624.88
RMSE for r=10: 624.87
RMSE for r=100: 624.84

Best r: 100 with RMSE: 624.84


# Question 5. RMSE spread


In [20]:
# List of seeds to test
seed_values = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# List to store the RMSE scores for each seed
rmse_scores = []

for seed in seed_values:
    # Split the data with the given seed
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=seed)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=seed)

    # Fill missing values with 0
    X_train_zero = X_train.fillna(0)
    X_val_zero = X_val.fillna(0)

    # Train a Linear Regression model without regularization
    model = LinearRegression()
    model.fit(X_train_zero, y_train)
    y_pred = model.predict(X_val_zero)

    # Calculate and store RMSE for the validation set
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_scores.append(rmse)
    print(f"Seed {seed} - RMSE: {round(rmse, 2)}")

# Calculate the standard deviation of the RMSE scores
std_rmse = round(np.std(rmse_scores), 3)
print(f"\nStandard deviation of RMSE scores: {std_rmse}")


Seed 0 - RMSE: 594.97
Seed 1 - RMSE: 566.04
Seed 2 - RMSE: 634.21
Seed 3 - RMSE: 571.4
Seed 4 - RMSE: 618.23
Seed 5 - RMSE: 534.85
Seed 6 - RMSE: 604.49
Seed 7 - RMSE: 666.14
Seed 8 - RMSE: 586.24
Seed 9 - RMSE: 622.51

Standard deviation of RMSE scores: 35.825


# Question 6. RMSE on Test

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import numpy as np

# Split the data into train (60%), validation (20%), and test (20%) sets using seed 9
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=9)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=9)

# Combine train and validation sets
X_train_combined = pd.concat([X_train, X_val])
y_train_combined = pd.concat([y_train, y_val])

# Fill missing values with 0
X_train_combined_zero = X_train_combined.fillna(0)
X_test_zero = X_test.fillna(0)

# Train a Ridge regression model with r = 0.001
model = Ridge(alpha=0.001)
model.fit(X_train_combined_zero, y_train_combined)

# Predict on the test set and calculate RMSE
y_pred_test = model.predict(X_test_zero)
rmse_test = round(np.sqrt(mean_squared_error(y_test, y_pred_test)), 2)

# Output the RMSE for comparison with the options
print("RMSE on the test dataset:", rmse_test)



RMSE on the test dataset: 535.08
